# (15.0) Описание решаемой задачи

Обновление ордерлога данными по найденным айсбергам и расчет метрик ликвидности на основании новых данных

# (15.1) Установка необходимых пакетов

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from matplotlib import pyplot as plt

# (15.2) Определение функций

In [2]:
# перевод времени в формат datetime
def ReverseToDateTime(t):
    format = '%Y%m%d%H%M%S%f'
    time_str = dt.datetime.strptime(t,format)
    return time_str

# восстановление стакана заявок и расчет основных метрик
def GlassBuild(for_lob):
    
    #формируем очередь заявок
    
    # устанавливаем маску лимитных заявок заявках (True - лимитная заявка)
    no_mkt_orders = for_lob['PRICE']!=0
    # таблица выставленных заявок
    plc_orders = for_lob[['ORDERNO','BUYSELL','PRICE','VOLUME']][(for_lob ['ACTION']==1) & no_mkt_orders]
    # таблица отмененных заявок
    wdr_orders = for_lob[['ORDERNO','VOLUME']][(for_lob ['ACTION'] == 0) & no_mkt_orders]
    # таблица исполненных заявок
    trd_orders = for_lob [['ORDERNO','VOLUME']][(for_lob['ACTION']==2) & no_mkt_orders]
    trd_orders = trd_orders.groupby(['ORDERNO']).sum()
    trd_orders['ORDERNO'] = trd_orders.index
    trd_orders.index=range(trd_orders.shape[0])
    # собираем все типы заявок в один датафрейм - очередь заявок
    lob_almost = pd.merge(plc_orders,wdr_orders, on='ORDERNO',how='left',suffixes=('_plc','_wdr'))
    lob_almost = pd.merge(lob_almost,trd_orders, on='ORDERNO', how='left')
    lob_almost = lob_almost.rename(columns={'VOLUME':'VOLUME_trd'})
    lob_almost = lob_almost.fillna(0)
    lob_almost['TOTAL_VOL'] = lob_almost['VOLUME_plc'] - lob_almost['VOLUME_wdr'] - lob_almost['VOLUME_trd']
    # удаляем из очереди заявок заявки с нулевым или отрицательным объемом
    lob_almost = lob_almost[['ORDERNO','BUYSELL','PRICE','VOLUME_plc','VOLUME_wdr','VOLUME_trd','TOTAL_VOL']][(lob_almost['TOTAL_VOL']>0)]
    
    # строим стакан
    
    lob = lob_almost.pivot_table(index = "PRICE", columns = "BUYSELL", values = "TOTAL_VOL", aggfunc = 'sum')
    lob.sort_index(axis = 0, inplace = True, ascending = False)
    lob = lob.fillna(0)
    
    # считаем метрики ликвидности
    
    # расчет лучшей цены на покупку
    bid_price = lob[lob['B'] > 0].index[0]
    # расчет объема на уровне лучшей цены на покупку
    bid_volume = lob['B'][bid_price]
    # расчет общего объема на стороне покупки
    bid_depth = lob['B'].sum()
    # расчет объема на уровне 10 лучших цен на стороне покупки
    b_10 = lob[lob['B'] > 0].index[0:10]
    volume_b_10 = 0
    for i in b_10:
        volume_b_10 = volume_b_10 + lob['B'][i]
    # расчет лучшей цены на продажу
    ask_price = lob[lob['S'] > 0].index[-1]
    # расчет объема на уровне лучшей цены на продажу
    ask_volume = lob['S'][ask_price]
    # расчет общего объема на стороне продажи
    ask_depth = lob['S'].sum()
    # расчет объема на уровне 10 лучших цен на стороне продажи
    s_10 = lob[lob['S'] > 0].index[-10:]
    volume_s_10 = 0
    for i in s_10:
        volume_s_10 = volume_s_10 + lob['S'][i]
    
    liquidity = [bid_price, bid_volume, bid_depth, volume_b_10, ask_price, ask_volume, ask_depth, volume_s_10]
     
    return(liquidity)

# (15.3) Список всех использующихся датафреймов¶¶

In [3]:
# orders - ордерлог на заданную дату
# iceberg_all - список всех найденных айсбергов
# orders_instr - ордерлог по инструменту
# ice_instr_day - список айсбергов по одному инструменту за один день
# chain_results - список всех orderno, которые были включены в цепочку с указанием цепочки, в которую они были включены
# all_day_chain - список всех цепочек с указанием их метрик

# (15.4) Определение входных параметров

In [4]:
# создаем список из инструментов, относящихся к числу голубых фишек
tickers = ['GAZP',
           'SBER',
           'GMKN',
           'LKOH',
           'MTSS',
           'MGNT',
           'TATN',
           'NVTK',
           'YNDX',
           'ROSN',
           'FIVE',
           'VTBR',
           'SNGS',
           'CHMF',
           'ALRS'
          ]

In [9]:
# создаем список из всех названий файлов с данными
orderlogs = ['OrderLog20190304.txt',
             'OrderLog20190305.txt',
             'OrderLog20190306.txt',
             'OrderLog20190307.txt',
             'OrderLog20190311.txt',
             'OrderLog20190312.txt',
             'OrderLog20190313.txt',
             'OrderLog20190314.txt',
             'OrderLog20190315.txt',
             'OrderLog20190318.txt',
             'OrderLog20190319.txt',
             'OrderLog20190320.txt',
             'OrderLog20190321.txt',
             'OrderLog20190322.txt',
             'OrderLog20190325.txt',
             'OrderLog20190326.txt',
             'OrderLog20190327.txt',
             'OrderLog20190328.txt',
             'OrderLog20190329.txt'
            ]

In [10]:
ticker = tickers[14] #ALRS
orderlog = orderlogs[0]
date = orderlog[8:16] #20190304
orderlog

'OrderLog20190304.txt'

# (15.5) Чтение данных

In [11]:
# читаем ордерлог
# orders - ордерлог на заданную дату
orders = pd.read_csv(orderlog, header = 0)
orders['together_time'] = str(date)+orders['TIME'].apply(str)
orders['TIME'] = [ReverseToDateTime(orders.together_time[i]) for i in range (len(orders.together_time))]
del orders['together_time']

In [12]:
# отбираем данные по инструменту
# orders_instr - ордерлог по инструменту
orders_instr = orders[orders.SECCODE == ticker]
orders_instr

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
240,241,ALRS,S,2019-03-04 10:00:00.000000,241,1,99.10,550,NaN,NaN
254,255,ALRS,S,2019-03-04 10:00:00.000000,255,1,98.85,33870,NaN,NaN
383,384,ALRS,S,2019-03-04 10:00:00.000000,384,1,99.76,50,NaN,NaN
388,389,ALRS,S,2019-03-04 10:00:00.000000,389,1,98.24,50,NaN,NaN
391,392,ALRS,B,2019-03-04 10:00:00.000000,392,1,92.61,1000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
5424123,5424124,ALRS,S,2019-03-04 18:44:02.288707,2546224,0,93.73,10,NaN,NaN
5424144,5424145,ALRS,B,2019-03-04 18:44:09.387865,2479540,0,93.04,500,NaN,NaN
5424145,5424146,ALRS,B,2019-03-04 18:44:09.469972,2479570,0,93.01,500,NaN,NaN
5424253,5424254,ALRS,S,2019-03-04 18:44:45.150707,1591060,0,94.17,370,NaN,NaN


In [15]:
# читаем файл с выявленными айсбергами
# iceberg_all - список всех айсбергов с указанием цепочки
iceberg_all = pd.read_csv('iceberg.csv', header = 0)
del iceberg_all['index']
iceberg_all.reset_index(level=0, inplace=True)
iceberg_all.rename(columns = {'index':'ID'}, inplace=True)

In [22]:
# ice_instr_day - список айсбергов по одному инструменту за один день
ice_instr_day = iceberg_all[(iceberg_all.SECCODE == ticker)&(iceberg_all.DATE == int(date))]

In [30]:
# читаем данные по цепочкам
# chain_results - список всех orderno, которые были включены в цепочку с указанием цепочки, в которую они были включены
chain_results = pd.read_csv('chain_results.csv', header = 0)
# читаем данные по основным характеристикам цепочек
# all_day_chain - список всех цепочек с указанием их метрик
all_day_chain = pd.read_csv('all_day_chain.csv', header = 0)

In [40]:
chain_results_instr_day = chain_results[(chain_results.TICKER == ticker)&(chain_results.DATE == int(date))].reset_index()
all_day_chain_instr_day = all_day_chain[(all_day_chain.TICKER == ticker)&(chain_results.DATE == int(date))].reset_index()

/Users/ekaterina/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [61]:
chain_results_instr_day.head(5)

,index,DATE,TICKER,ORDERNO,CHAIN_ID
0,2163,20190304,ALRS,226531,226531
1,2164,20190304,ALRS,355352,355352
2,2165,20190304,ALRS,822918,822918
3,2166,20190304,ALRS,822919,822919
4,2167,20190304,ALRS,822920,822919


In [65]:
test = all_day_chain_instr_day.head(5)
test

,index,DATE,TICKER,CHAIN_ID,LENGTH,VOLUME,PRICE,LIFE,VALUE,VOLUME_INI,VALUE_INI,HID/ALL
0,1575,20190304,ALRS,226531,1,8660,94.03,0 days 02:38:58.049992000,814299.8,1000.0,94030.0,0.115473
1,1576,20190304,ALRS,355352,1,3610,95.40,0 days 00:00:02.587567000,344394.0,1000.0,95400.0,0.277008
2,1577,20190304,ALRS,822918,1,400,95.31,0 days 00:00:00.000000000,38124.0,250.0,23827.5,0.625000
3,1578,20190304,ALRS,822919,2,2260,95.31,0 days 00:00:00.000000000,215400.6,100.0,9531.0,0.044248
4,1579,20190304,ALRS,822921,1,400,95.31,0 days 00:00:00.000000000,38124.0,90.0,8577.9,0.225000


In [ ]:
%%time
# определяем ордерно всех заявок из цепочки
# находим эти заявки в orders_instr
# обновляем первоначальный объем первой заявки в цепочке
# удалить запись о выставлении всех остальных заявок в цепочке
# обновляем ордерно в указании об исполнении всех других заявок в цепочке на ордерно первой заявки в цепочке
for a in range(len(all_day_chain_instr_day)):
    
    chain_id = all_day_chain_instr_day.loc[a, 'CHAIN_ID']
    print(chain_id)
    orders_in_chain = chain_results_instr_day[chain_results_instr_day.CHAIN_ID == chain_id]['ORDERNO'].to_list()
    print(chain_id, orders_in_chain)
    first_order_in_chain = orders_in_chain[0]
    # ищем первый элемент цепочки и обновляем объем на величину общего объема цепочки
    index_in_all_day = all_day_chain_instr_day.index[all_day_chain_instr_day.CHAIN_ID == chain_id].to_list()[0]
    chain_volume = all_day_chain_instr_day.loc[index_in_all_day, 'VOLUME']
    orders_instr.loc[(orders_instr['ORDERNO'] == first_order_in_chain) & (orders_instr['ACTION'] == 1),'VOLUME'] = chain_volume
        
    # если длина цепочки больше единицы, то 1) удаляем все записи о выставлении последующих элементов цепочки
    # 2) меняем ордерно об исполнении или отмене последующих элементов цепочки на ордерно первого элемента цепочки
    if len(orders_in_chain) != 1:
        
        print(chain_id, orders_in_chain,1)
        # исключаем из цепочки первый элемент
        rest_orders_in_chain = orders_in_chain[1:len(orders_in_chain)]
        # удаляем записи о выставлении последующих элементов цепочки
        orders_instr = orders_instr.drop(orders_instr[(orders_instr['ORDERNO'].isin(rest_orders_in_chain)) & (orders_instr['ACTION']==1)].index)
        # меняем ордерно об исполнении или отмене последующих элементов цепочки на ордерно первого элемента
        orders_instr.loc[(orders_instr['ORDERNO'].isin(rest_orders_in_chain)) & (orders['ACTION']!=1),'ORDERNO'] = first_order_in_chain


226531
226531 [226531]
355352
355352 [355352]
822918
822918 [822918]
822919
822919 [822919, 822920]
822919 [822919, 822920] 1
822921
822921 [822921]
822924
822924 [822924]
844873
844873 [844873]
896774
896774 [896774]
896775
896775 [896775, 896779]
896775 [896775, 896779] 1
898289
898289 [898289]
898320
898320 [898320]
898369
898369 [898369]
898483
898483 [898483]
898619
898619 [898619]
900206
900206 [900206]
906298
906298 [906298]
908049
908049 [908049, 908049, 908178, 908335]
908049 [908049, 908049, 908178, 908335] 1
908049
908049 [908049, 908049, 908178, 908335]
908049 [908049, 908049, 908178, 908335] 1
916526
916526 [916526]
916926
916926 [916926]
919623
919623 [919623]
1072610
1072610 [1072610]
1077833
1077833 [1077833]
1099220
1099220 [1099220, 1100047, 1100055]
1099220 [1099220, 1100047, 1100055] 1
1111806
1111806 [1111806]
1112085
1112085 [1112085, 1112133]
1112085 [1112085, 1112133] 1
1112523
1112523 [1112523]
1115879
1115879 [1115879]
1115880
1115880 [1115880]
1115884
1115884

In [60]:
orders_instr[(orders_instr.ORDERNO == 226531)|
             (orders_instr.ORDERNO == 355352)|
             (orders_instr.ORDERNO == 822918)|
             (orders_instr.ORDERNO == 822919)|
             (orders_instr.ORDERNO == 822920)]

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
396122,396123,ALRS,B,2019-03-04 10:09:44.700009,226531,1,94.03,8660,NaN,NaN
655558,655559,ALRS,B,2019-03-04 10:29:36.302328,355352,1,95.40,3610,NaN,NaN
655899,655900,ALRS,B,2019-03-04 10:29:38.889895,355352,2,95.40,3610,2.935661e+09,95.40
2253046,2253047,ALRS,B,2019-03-04 12:48:42.750001,226531,2,94.03,8660,2.935785e+09,94.03


In [43]:
chain_id = chain_results_instr_day.loc[0, 'CHAIN_ID']

In [44]:
chain_results_instr_day[chain_results_instr_day.CHAIN_ID == chain_id]['ORDERNO'].to_list()

[226531]

In [56]:
index_in_all_day = all_day_chain_instr_day.index[all_day_chain_instr_day.CHAIN_ID == chain_id].to_list()[0]
chain_volume = all_day_chain_instr_day.loc[index_in_all_day, 'VOLUME']
chain_volume

8660

In [51]:
all_day_chain_instr_day.index[all_day_chain_instr_day.CHAIN_ID == chain_id]

TypeError: 'RangeIndex' object is not callable

In [56]:
# устанавливаем маску участия в цепочке (True - входит в цепочку)
chain = iceberg_all['GROUP_ID'].notnull()
# chained - айсберги, которые были объединены в цепочки
chained = iceberg_all[['DATE','ORDERNO','TOTAL_VOLUME','GROUP_ID','ID']][(chain == True)]
# not_chained - айсберги, которые не были объединены в цепочки
not_chained = iceberg_all[['DATE','ORDERNO','VOLUME','VOLUME_INI','ID']][(chain == False)]

In [253]:
%%time
# если айсберг-заявка не была объединена в цепочку, то для обновления ордерлога необходимо просто по 
# номеру заявки-дате найти соответствующую заявку с action = 1 в ордерлоге и обновить объем
# на величину (VOLUME_INI - VOLUME)
for l in orderlogs:

    date = int(l[8:16])
    orders = pd.read_csv(l, header = 0)
    icebergs = not_chained[not_chained.DATE == date].reset_index()
    
    for i in range(len(icebergs)):
        
        orderno = icebergs.loc[i,'ORDERNO']
        orders.loc[(orders['ORDERNO']==orderno) & (orders['ACTION']==1),'VOLUME'] -= icebergs.loc[i,'VOLUME']
        
    orderlog_name = '/Users/ekaterina/Desktop/w/курсовая (2)/data/updated/' + l       
    orders.to_csv(orderlog_name,index = False)

CPU times: user 20min 31s, sys: 2min 24s, total: 22min 55s
Wall time: 17min 52s


In [244]:
orders[orders.ORDERNO == 199077]

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
342352,342353,GAZP,B,100643595004,199077,1,156.28,4830,NaN,NaN
394216,394217,GAZP,B,100935807572,199077,2,156.28,550,2.935640e+09,156.28
394229,394230,GAZP,B,100935808904,199077,2,156.28,500,2.935640e+09,156.28
394266,394267,GAZP,B,100935902211,199077,2,156.28,10,2.935640e+09,156.28
394270,394271,GAZP,B,100935906230,199077,2,156.28,760,2.935640e+09,156.28
394341,394342,GAZP,B,100936427550,199077,2,156.28,10,2.935640e+09,156.28
394358,394359,GAZP,B,100936467866,199077,2,156.28,3000,2.935640e+09,156.28


In [236]:
test1 = not_chained[not_chained.DATE == 20190304].reset_index()
test1.loc[0,'ORDERNO']

199077

In [235]:
test3 = not_chained[not_chained.ORDERNO == test1.loc[0,'ORDERNO']]
test3

,DATE,ORDERNO,VOLUME,VOLUME_INI,ID
0,20190304,199077,-2830,2000,0


In [234]:
test = pd.read_csv('OrderLog20190304.txt', header = 0)
test2 = test[(test.ORDERNO == test1.loc[0,'ORDERNO'])]
test2

,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE
342352,342353,GAZP,B,100643595004,199077,1,156.28,2000,NaN,NaN
394216,394217,GAZP,B,100935807572,199077,2,156.28,550,2.935640e+09,156.28
394229,394230,GAZP,B,100935808904,199077,2,156.28,500,2.935640e+09,156.28
394266,394267,GAZP,B,100935902211,199077,2,156.28,10,2.935640e+09,156.28
394270,394271,GAZP,B,100935906230,199077,2,156.28,760,2.935640e+09,156.28
394341,394342,GAZP,B,100936427550,199077,2,156.28,10,2.935640e+09,156.28
394358,394359,GAZP,B,100936467866,199077,2,156.28,3000,2.935640e+09,156.28


In [256]:
%%time
# если айсберг-заявка была объединена в цепочку, то для обновления ордерлога необходимо обновить первоначальный объем первой заявки в цепочке,
# удалить запись о выставлении
for l in orderlogs:

    date = int(l[8:16])
    orderlog_name = '/Users/ekaterina/Desktop/w/курсовая (2)/data/updated/' + l
    orders = pd.read_csv(orderlog_name, header = 0)
    icebergs = chained[chained.DATE == date].reset_index()
    
    for i in range(len(icebergs)):
        
        orderno = icebergs.loc[i,'ORDERNO']
        
        if icebergs.loc[i,'ID'] == icebergs.loc[i,'GROUP_ID']: # если это первый элемент цепочки айсберга, то ему необходимо перезаписать объем на объем цепочки
            
            first_element_id = icebergs.loc[i,'ORDERNO']
            orders.loc[(orders['ORDERNO']==orderno) & (orders['ACTION']==1),'VOLUME'] = icebergs.loc[i,'TOTAL_VOLUME']
            
        else: # если это не первый элемент цепочки айсберга, то ему необходимо:
            
            orders = orders.drop(orders[(orders['ORDERNO']==orderno) & (orders['ACTION']==1)].index) # удалить выставленные заявки
            orders.loc[(orders['ORDERNO']==orderno) & (orders['ACTION']!=1),'ORDERNO'] = first_element_id # поменять номер заявки сделки и отмены на номер первой заявки в цепочке
               
    orders.to_csv(orderlog_name,index = False)

KeyboardInterrupt: 